## Seleksi Fitur ##
   Seleksi fitur adalah teknik untuk memilih fitur penting dan relevan terhadap data dan mengurangi fitur yang tidak relevan. Seleksi fitur bertujuan untuk memilih fitur terbaik dari suatu kumpulan data fitur. Seleksi fitur bertujuan untuk memilih fitur terbaik dari suatu kumpulan data fitur. Tujuan dari penelitian ini adalah menerapkan metode Information Gain dalam sistem seleksi fitur untuk Permasalahan cuaca . Metode Information Gain adalah metode yang menggunakan teknik scoring untuk pembobotan sebuah fitur dengan menggunakan maksimal entropy. Fitur yang dipilih adalah fitur dengan nilai Information Gain yang lebih besar atau sama dengan nilai threshold tertentu.

In [40]:
from scipy import stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import math

In [41]:
df = pd.read_csv('feature selection1.csv')
df

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


### mencari entropy ###
Untuk menghitung Information gain perlu dihitung dahulu nilai informasi dalam suatu bits dari suatu kumpulan obyek. Cara penghitungan dilakukan dengan menggunakan konsep entropi. Entropi menyatakan impurity suatu kumpulan obyek . Berikut merupakan definisi dari entropi suatu ruang sampel data (S):
$$ E(T) = \sum_{i=1}^n {-P_i\log{P_i}} $$

In [42]:
def banyak_elemen (kolom, data):
    kelas=[]
    for i in range (len(data)):
        if data.values.tolist()[i][kolom] not in kelas:
            kelas.append(data.values.tolist()[i][kolom])
    return kelas

In [43]:
def entropy(T):
    hasil=0
    jumlah=0
    for y in T:
        jumlah+=y
    for z in range (len(T)):
        if jumlah!=0:
            T[z]=T[z]/jumlah

    for i in T:
        if i != 0:
            hasil-=i*math.log(i,2)
    return hasil

In [44]:
def probJenis(jenis,kelas,kolomJenis,kolomKelas,data):
    hasil=[]
    for x in range (len(kelas)):
        hasil.append(0)
    for i in range (len(data)):
        if data.values.tolist()[i][kolomJenis] == jenis:
            for j in range (len(kelas)):
                if data.values.tolist()[i][kolomKelas] == kelas[j]:
                    hasil[j]+=1
    return hasil

In [45]:
def t_list(atribut,n):
    temp=[]
    for i in range(len(atribut)):
        temp.append(probJenis(atribut[i],kelas,df.shape[1]-n,df.shape[1]-1,df))
    return temp

In [46]:
def e_list(atribut,n):
    temp=[]
    tx=t_list(atribut,n)
    for i in range (len(atribut)):
        ent=entropy(tx[i])
        temp.append(ent)
    return temp

In [47]:
def probKelas(kelas,kolomKelas,data):
    hasil=[]
    for x in range(len(kelas)):
        hasil.append(0)
    for i in range (len(data)):
        for j in range (len(kelas)):
            if data.values.tolist()[i][kolomKelas] == kelas[j]:
                hasil[j]+=1
    return hasil

### Gain ###
Gain adalah sebuah fiktur yang terdapat pada sebuah data , untuk menghitungnya contoh rumusnya : $$ \operatorname{Gain}(T, X) = \operatorname{Entropy}(T) - \sum_{v\in{T}} \frac{T_{X,v}}{T} E(T_{X,v}) $$

In [48]:
def gain(T_list,E_list,pKelas,data):
    txv=0
    for i in range (len(E_list)):
        jumlah=0
        for y in range (len(T_list[i])):
            jumlah+=T_list[i][y]
        txv+=jumlah/(len(data))*(E_list[i])
    eUtama=entropy(pKelas)
    hasil=eUtama-txv
    return hasil

In [49]:
kelas=banyak_elemen(df.shape[1]-1,df)
outlook=banyak_elemen(df.shape[1]-5,df)
temp=banyak_elemen(df.shape[1]-4,df)
humidity=banyak_elemen(df.shape[1]-3,df)
windy=banyak_elemen(df.shape[1]-2,df)

In [50]:
pKelas=probKelas(kelas,df.shape[1]-1,df)

In [51]:
tOutlook=t_list(outlook,5)
tTemp=t_list(temp,4)
tHum=t_list(humidity,3)
tWin=t_list(windy,2)

In [52]:
eOutlook=e_list(outlook, 5)
eTemp=e_list(temp, 4)
eHum=e_list(humidity, 3)
eWin=e_list(windy, 2)

In [53]:
from IPython.display import HTML, display
import tabulate
table=[
    ["attributes"]+["Jenis"]+["no"]+["yes"]+["entropy"]+["gain"],
    ["OUTLOOK"]+["Sunny"]+[tOutlook[0][0]]+[tOutlook[0][1]]+[eOutlook[0]]+[gain(tOutlook,eOutlook,pKelas,df)],
    [""]+["Overcast"]+[tOutlook[1][0]]+[tOutlook[1][1]]+[eOutlook[1]]+[],
    [""]+["Rainy"]+[tOutlook[2][0]]+[tOutlook[2][1]]+[eOutlook[2]]+[],
    ["TEMPERATURE"]+["Hot"]+[tTemp[0][0]]+[tTemp[0][1]]+[eTemp[0]]+[gain(tTemp,eTemp,pKelas,df)],
    [""]+["Mild"]+[tTemp[1][0]]+[tTemp[1][1]]+[eTemp[1]]+[],
    [""]+["Cool"]+[tTemp[2][0]]+[tTemp[2][1]]+[eTemp[2]]+[],
    ["HUMIDITY"]+["High"]+[tHum[0][0]]+[tHum[0][1]]+[eHum[0]]+[gain(tHum,eHum,pKelas,df)],
    [""]+["Normal"]+[tHum[1][0]]+[tHum[1][1]]+[eHum[1]]+[],
    ["WINDY"]+["False"]+[tWin[0][0]]+[tWin[0][1]]+[eWin[0]]+[gain(tWin,eWin,pKelas,df)],
    [""]+["True"]+[tWin[1][0]]+[tWin[1][1]]+[eWin[1]]+[],
    ]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

attributes,Jenis,no,yes,entropy,gain
OUTLOOK,Sunny,3,2,0.9709505944546686,0.2467498197744391
,Overcast,0,4,0.0,
,Rainy,2,3,0.9709505944546686,
TEMPERATURE,Hot,2,2,1.0,0.029222565658954647
,Mild,2,4,0.9182958340544896,
,Cool,1,3,0.8112781244591328,
HUMIDITY,High,4,3,0.9852281360342516,0.15183550136234136
,Normal,1,6,0.5916727785823275,
WINDY,False,2,6,0.8112781244591328,0.04812703040826927
,True,3,3,1.0,
